<a href="https://colab.research.google.com/github/brianferrell787/Classifying-community-engaged-research-with-transformer-based-models/blob/main/ClassificationProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Upload the two training data files, and hold out dataset**


In [1]:
#Upload zip file turned in
from google.colab import files
uploaded = files.upload()

Saving DataforProject.zip to DataforProject.zip


In [ ]:
!unzip "/content/DataforProject.zip"

# **Install packages and other various tools**

In [3]:
#GPU information 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jan  5 14:18:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
pip install simpletransformers

     |████████████████████████████████| 204kB 12.6MB/s 
     |████████████████████████████████| 1.8MB 21.8MB/s 
     |████████████████████████████████| 317kB 60.3MB/s 
     |████████████████████████████████| 7.4MB 61.7MB/s 
     |████████████████████████████████| 1.1MB 67.9MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 1.5MB 50.7MB/s 
     |████████████████████████████████| 2.9MB 51.1MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 133kB 57.5MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 163kB 50.2MB/s 
     |████████████████████████████████| 112kB 69.9MB/s 
     |████████████████████████████████| 4.5MB 65.9MB/s 
     |████████████████████████████████| 890kB 67.2MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 122kB 68.7MB/s

In [6]:
# WandB – Install the W&B library -- You don't need this unless you want to see the wandb graph but you have to create a account 
!pip install wandb --upgrade

Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (0.10.12)


In [1]:
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### **Upload JSON file and Excel file for training data**

In [2]:
train1_df = pd.read_excel('training_df.xlsx')
train1_df=train1_df.fillna("")
train1_df = shuffle(train1_df)
train1_df

,AimsGoal,Hypothesis,Study Design,Background,Class
2106,building capacity for children ' s involvement...,"this study will use an interpretivist , qualit...",study design : this is a qualitative interpret...,despite legislative initiatives supporting chi...,5
1709,whole - school learning in an urban middle sch...,this survey will examine the process and resul...,"in keeping with the nature of the data , the c...",the initiative was instigated by a proposal ma...,5
703,implementation and evaluation of a 24 - hour a...,secondly : it will be important to establish a...,"design : prospective , single - arm , non - ra...",hypertension ( htn ) is the leading mortality ...,3
2613,inclusion project : surveys to students in vir...,h1 : the inclusion project will raise awarenes...,"in conjunction with project staff , the evalua...",inclusion project is a multi - phase project f...,5
1703,inter - school literacy in an urban middle sch...,this article will examine the process and effe...,"in keeping with the nature of the data , the c...",the study was instigated by a request made by ...,5
...,...,...,...,...,...
1548,to end consumer health outreach and project ac...,participants through online application resear...,consistent with a community - based participat...,"at 12 % proficiency , u . s . health literacy ...",5
1105,relationship between exposure to anesthetic dr...,multiple studies have been conducted examining...,we plan on examining the school enrollment rec...,fatalities resulting from drug overdose have d...,4
1364,the impact of human - animal interaction on ad...,hypothesis : aims 2 . we hypothesize that chil...,breif overview of method for each aim ( detail...,alzheimers and related diseases data shows an ...,4
1912,inclusion projects : outreach to schools in vi...,h1 : the action project will raise awareness i...,"in conjunction with project staff , the evalua...",integration project is a three - year project ...,5


In [3]:
train2_df = pd.read_json('/content/training_df7k.json')
train2_df = shuffle(train2_df)
train2_df

,Text,Class
6124,to end consumer literacy outreach and project ...,5
4029,"community engagement education program - 2 , a...",2
3290,examples of effective research about mental il...,2
107,incentives for primary care use : a randomized...,0
5651,agri - cultureo : growing food and community w...,4
...,...,...
2001,,0
4266,assessment of health needs : a community based...,2
516,enhancing at - risk latina participant ' s pra...,0
3916,level of awareness for students and adolescent...,2


## **Combine columns for the second training set to match first and combine both dataframes for more data**


I have two files for training data from augmenting it so that is why I am combining them

In [4]:
#This adds the columns
train1_df['Text'] = train1_df['AimsGoal'] + train1_df['Hypothesis'] + train1_df['Background'] + train1_df['Study Design'] 


#This creates a new dataframes of the labels and train_df2 which will later be combined with the other training set just to have more data.. it is not necessary 
classes = pd.DataFrame(train1_df['Class'])
trainingset_2 = pd.DataFrame(train1_df['Text'])
trainingset_2['Class'] = classes

trainingset_2

,Text,Class
2106,building capacity for children ' s involvement...,5
1709,whole - school learning in an urban middle sch...,5
703,implementation and evaluation of a 24 - hour a...,3
2613,inclusion project : surveys to students in vir...,5
1703,inter - school literacy in an urban middle sch...,5
...,...,...
1548,to end consumer health outreach and project ac...,5
1105,relationship between exposure to anesthetic dr...,4
1364,the impact of human - animal interaction on ad...,4
1912,inclusion projects : outreach to schools in vi...,5


In [5]:
train_df = pd.concat([trainingset_2, train2_df], axis=0, sort=False)
train_df

,Text,Class
2106,building capacity for children ' s involvement...,5
1709,whole - school learning in an urban middle sch...,5
703,implementation and evaluation of a 24 - hour a...,3
2613,inclusion project : surveys to students in vir...,5
1703,inter - school literacy in an urban middle sch...,5
...,...,...
2001,,0
4266,assessment of health needs : a community based...,2
516,enhancing at - risk latina participant ' s pra...,0
3916,level of awareness for students and adolescent...,2


In [6]:
unique = train_df.groupby('Class').nunique()

print (unique)

       Text
Class      
0      1931
1      1427
2      1413
3      1564
4      1431
5      1404


## **Find and replace Nan/NULL values**


In [7]:
train_df=train_df.fillna("")


In [8]:
train_df.isnull().any()

Text     False
Class    False
dtype: bool

## **Change the training values to a less rigid classification spectrum(n = 3 intead of 5)**


In [9]:
train_df['Class'] = train_df['Class'].replace([2],1)
train_df['Class'] = train_df['Class'].replace([3],2)
train_df['Class'] = train_df['Class'].replace([5],4)
train_df['Class'] = train_df['Class'].replace([4],3)
train_df['Class'] = train_df['Class'].replace([3],2)
train_df

,Text,Class
2106,building capacity for children ' s involvement...,2
1709,whole - school learning in an urban middle sch...,2
703,implementation and evaluation of a 24 - hour a...,2
2613,inclusion project : surveys to students in vir...,2
1703,inter - school literacy in an urban middle sch...,2
...,...,...
2001,,0
4266,assessment of health needs : a community based...,1
516,enhancing at - risk latina participant ' s pra...,0
3916,level of awareness for students and adolescent...,1


## **Preproccessing steps(cleaning, lemmatizing, stopword removal, padding, tokenization, etc**


I also have to combine the columns for the hold out data

In [10]:
import nltk
import re  
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    nltk.download('stopwords')
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", "  ", text)
    text = re.sub(r"=", "  ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", "  ", text)
    text = re.sub(r";", "  ", text)
    text = re.sub(r"http", "  ", text)
    text = re.sub(r"www", "  ", text)
    text = re.sub(r"-", "  ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\d+", " ", text)

    ## Lemmatize
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemma_words)

    return text


In [12]:
# apply the above function to df['text']
traintext = train_df['Text'].map(lambda x: clean_text(x))

Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-dat

In [13]:
traintext

2106    building capacity child involvement child welf...
1709    whole school learning urban middle school stud...
703     implementation evaluation hour ambulatory bloo...
2613    inclusion project survey student virginia help...
1703    inter school literacy urban middle school stud...
                              ...                        
2001                                                     
4266    assessment health need community based approac...
516     enhancing risk latina participant practice gen...
3916    level awareness student adolescent autism atte...
4960    patient perception compliance package usage co...
Name: Text, Length: 9412, dtype: object

In [14]:
test_df = pd.read_excel('Hold Out data.xlsx' )
test_df =test_df.fillna("")
test_df['New'] = test_df['AimsGoal'] + test_df['Hypothesis'] + test_df['Background'] + test_df['Study Design']
test_df['New'] =test_df['New'].astype(str)
valtext = test_df['New'].map(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

## **Turning test set into 3 class spectrum**

In [15]:
test_df['Class'] = test_df['Class'].replace([2],1)
test_df['Class'] = test_df['Class'].replace([3],2)
test_df['Class'] = test_df['Class'].replace([5],4)
test_df['Class'] = test_df['Class'].replace([4],3)
test_df['Class'] = test_df['Class'].replace([3],2)
test_df

,ID,AimsGoal,Hypothesis,Background,Study Design,Class,New
0,HM20013139,Identifying and Prioritizing Patient Goals Bef...,The primary aim of this pilot feasibility stud...,Total Knee Replacement (TKR) surgery is consid...,Design Overview:Pilot feasibility study Sample...,0,Identifying and Prioritizing Patient Goals Bef...
1,HM20003563,The Impact of Home Modifications on the Safety...,Participants will report increased performance...,The vast majority of United States homeowners ...,The study will be conducted in collaboration w...,2,The Impact of Home Modifications on the Safety...
2,HM20014813,"Cultural Diversity Within Schools Survey , The...",The research questions guiding this study are:...,The purpose of this survey is to better unders...,Study design: The proposed study will use a su...,2,"Cultural Diversity Within Schools Survey , The..."
3,HM20010722,Stakeholder Perspectives on Social Needs Scree...,Research questionsHow do parents of children w...,Approximately 10% of children under age 5 have...,"The overarching, tiered approach to identifica...",0,Stakeholder Perspectives on Social Needs Scree...
4,HM20011134,Development of Psychosocial Need Assessment fo...,"The proposed study will develop a reliable, va...",Cystic fibrosis (CF) places children and adole...,"The study will be longitudinal in nature, cons...",0,Development of Psychosocial Need Assessment fo...
...,...,...,...,...,...,...,...
304,HM20002308,Virginia's National Core Indicator Project Sec...,.General research questions will include:ò Wha...,National Core Indicators (NCI) is a collaborat...,This study will be a secondary data analysis o...,1,Virginia's National Core Indicator Project Sec...
305,HM15211,Identifying Medication-Related Issues with Tra...,"Dominion Place, located in Richmond, Virginia,...",,Background information on transitions of care ...,2,Identifying Medication-Related Issues with Tra...
306,HM20003653,Reporting Intimate Violence to Law Enforcement...,Little is known about when and how African Ame...,The proposed study focuses on developing multi...,To address the dearth of knowledge of African ...,1,Reporting Intimate Violence to Law Enforcement...
307,HM20014642,Barriers to care in the diagnosis and treatmen...,We hypothesize that women who have barriers to...,Despite improved screening programs and vaccin...,"This is a descriptive, observational study inv...",1,Barriers to care in the diagnosis and treatmen...


## **Last few steps to format the data into what SimpleTransformer's needs**

In [16]:
training = pd.DataFrame(traintext)
training['Labels'] = train_df['Class']
training

,Text,Labels
2106,building capacity child involvement child welf...,2
1709,whole school learning urban middle school stud...,2
703,implementation evaluation hour ambulatory bloo...,2
2613,inclusion project survey student virginia help...,2
1703,inter school literacy urban middle school stud...,2
...,...,...
2001,,0
4266,assessment health need community based approac...,1
516,enhancing risk latina participant practice gen...,0
3916,level awareness student adolescent autism atte...,1


**The dataframe below which was previously uploaded is the testing data and the hold out data combined; therefore, I only use the first 280 values as the other 30 are used as the hold out dataset.**

In [17]:
testing = pd.DataFrame(valtext)
testing['Labels'] = test_df['Class']
test = testing[:279]
test

,New,Labels
0,identifying prioritizing patient goal total kn...,0
1,impact home modification safety community dwel...,2
2,cultural diversity within school survey resear...,2
3,stakeholder perspective social need screening ...,0
4,development psychosocial need assessment cysti...,0
...,...,...
274,comprehensive injury surveillance latin americ...,1
275,phase drug interaction self administration stu...,0
276,exploration afrocentric mentoring paradigm sch...,2
277,resident staff perception safety engagement co...,2


In [18]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['Class']),train_df['Class'])
class_weights = dict(enumerate(class_weights))
class_weights

{0: 1.5304065040650408, 1: 1.0844567346468488, 2: 0.7020213321399269}

In [20]:
#This code will log you in to the wandb website but you will need an account for it. It is not neccesary for training though, just visualizes training
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [21]:
from simpletransformers.classification import ClassificationModel


#This will probably train for ~1 hour
# Training arguments
model_args = {
    'evaluate_during_training': True,
    'logging_steps': 10,
    'num_train_epochs': 3,
    'evaluate_during_training_steps': 500,
    'save_eval_checkpoints': False,
    #"save_steps": 109,
    'manual_seed':4,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'learning_rate': 3e-5,
    'sliding_window':True,
    'max_seq_length':128,
    'use_cuda':True,
    'silent':True, #Silent means no verbose
    'no_cache':True,



  #wandb_project is only needed if you do the wandb login
   # 'wandb_project': "ProjectDemonstration",
}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", 
    "bert-base-uncased",
    num_labels=3, #this would be 6 if we were using the 6 class spectrum 
    weight=[1.5304065040650408,1.0844567346468488, 0.7020213321399269],
    args=model_args
)

# Train model
model.train_model(training,eval_df=test)


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:391: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:970: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(14442,
 {'eval_loss': [0.3986532556406443,
   0.2567031302491603,
   0.12695340983332679,
   0.14006401783276987,
   0.14031018596283937,
   0.10760424066872663,
   0.13278462265757443,
   0.11756941716026477,
   0.1960235683784094,
   0.09699081107316902,
   0.102783510326748,
   0.15117388670582374,
   0.15712155782004997,
   0.13345776799203954,
   0.1442823945233536,
   0.1287712268310055,
   0.13535876698668212,
   0.10876490533992311,
   0.11661123323472232,
   0.11215155005952462,
   0.13495525657246019,
   0.08840198760338935,
   0.11945976741775692,
   0.10699277952845757,
   0.10404895688698611,
   0.12538172642243067,
   0.13657752011247523,
   0.11536818147810776,
   0.10950265635892961,
   0.10953917424972566,
   0.10982202144185407],
  'global_step': [500,
   1000,
   1500,
   2000,
   2500,
   3000,
   3500,
   4000,
   4500,
   4814,
   5000,
   5500,
   6000,
   6500,
   7000,
   7500,
   8000,
   8500,
   9000,
   9500,
   9628,
   10000,
   10500,
   11000,
   11500

In [22]:
result, model_outputs, wrong_predictions = model.eval_model(test)
result

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:970: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


{'eval_loss': 0.10982202144185407, 'mcc': 1.0}

The output below is the predictions on the original/testing data, and the last 30 values are the hold out data which is where I get my hold out dataset acc. from 


I make predictions based on the bestmodel output and the regular model output to compare. Which is provided by SimpleTransformers under the "outputs" folder found if you click the files buttom on the left side pane

In [23]:
prediction_v1, raw_outputs = model.predict(valtext)
prediction_v1 

array([0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 0, 1, 2, 2, 0, 2, 1, 0,
       1, 2, 0, 1, 2, 0, 2, 0, 0, 1, 0, 2, 0, 1, 2, 2, 2, 2, 1, 2, 1, 2,
       1, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0,
       1, 2, 2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       0, 0, 2, 0, 2, 2, 0, 2, 2, 1, 1, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2, 2,
       0, 0, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 0,
       0, 1, 0, 0, 2, 0, 1, 1, 2, 1, 2, 0, 2, 2, 0, 2, 1, 2, 1, 1, 1, 2,
       1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 0, 2, 1, 2, 2, 1, 2, 0,
       2, 1, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 1, 2, 0,
       2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2,
       0, 2, 2, 1, 0, 2, 2, 0, 2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 0, 0, 2, 1, 2,
       2, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1,
       2, 0, 0, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 0,

In [24]:
bestcheckpoint_model = ClassificationModel("bert", "outputs/best_model/",args=model_args,num_labels=3)
predictions, raw_outputs = bestcheckpoint_model.predict(valtext)
print(predictions)

[0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 0, 1, 2, 2, 0, 2, 1, 0, 1, 2, 0, 1, 2, 0, 2, 0, 0, 1, 0, 2, 0, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 1, 1, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2, 2, 0, 0, 1, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 0, 0, 1, 0, 0, 2, 0, 1, 1, 2, 1, 2, 0, 2, 2, 0, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 0, 2, 1, 2, 2, 1, 2, 0, 2, 1, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 0, 2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 2]


I usually copy and paste the predictions in a google sheet I created that has all of the original data and compare my models from that but for this I can show the F1 score from this code file below:

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

#f1 score of the last 30 values as they are the hold out data
#I print out for each class and the average
scores=f1_score(testing[279:]['Labels'], prediction_v1[279:], average=None)
scoremacro=f1_score(testing[279:]['Labels'], prediction_v1[279:], average='macro')



print("F1 scores per class: ") 
print("")
print("Zero's class: ", scores[0])
print("One's class: ", scores[1])
print("Two's class: ", scores[2])
print("")
print("Macro F1 score: ")
print("")
print("F1 macro :",scoremacro)

# **The cross-validation models were performed using this**

In [ ]:
from sklearn.model_selection  import StratifiedKFold, cross_val_score
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
import pandas as pd
from simpletransformers.classification import ClassificationModel
import logging


# Training arguments
model_args = {
    'evaluate_during_training': False,
    'logging_steps': 10,
    'num_train_epochs': 3,
    'evaluate_during_training_steps': 500,
    'save_eval_checkpoints': False,
    #"save_steps": 109,
    'manual_seed':4,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'learning_rate': 3e-5,
    'sliding_window':True,
    'max_seq_length':128,
    'use_cuda':True,
    'silent':True,
    'no_cache':True,



  
    #'wandb_project': "xlmroberta",
}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", 
    "bert-base-uncased",
    num_labels=3,
    weight=[1.5304065040650408,1.0844567346468488, 0.7020213321399269],
    args=model_args)



#prepare cross validation
n=8
kf = KFold(n_splits=n, random_state=42, shuffle=True)

results = []
# Log scores
#for cv_score in results:
  # wandb.log({'score': results})
  # wandb.watch(results, log='all')

for train_index, val_index in kf.split(training):
  # splitting Dataframe (dataset not included)
    train = training.iloc[train_index]
    val = training.iloc[val_index]
  # train the model
    model.train_model(train)
  # validate the model 
    result, model_outputs, wrong_predictions = model.eval_model(val, acc=accuracy_score)
    print(result['acc'])
  # append model score
    results.append(result['acc'])


print("results",results)
print(f"Mean-Precision: {sum(results) / len(results)}")

In [ ]:
model.train_model(training)

In [ ]:
predictions2, raw_outputs2 = model.predict(valtext)
print(predictions2)